In [8]:

import numpy as np
import math

L, d_k, d_v = 4, 8, 8
q = np.random.randn(L, d_k)
k = np.random.randn(L, d_k)
v = np.random.randn(L, d_v)

In [9]:
print("Q\n", q)
print("K\n", k)
print("V\n", v)

Q
 [[ 1.06265353 -1.04154709  0.31903798 -1.97891434  0.37866731 -0.49480558
   0.14147369 -0.38059492]
 [ 0.15804184 -0.53026353  0.368618   -1.09082603  0.56294831  1.03452456
  -2.03631891  1.41276778]
 [ 1.09129202  0.52080012 -0.24924261 -0.45375836 -0.18950295 -1.42962098
  -0.5359392   0.44610144]
 [ 0.4854946  -0.0640484  -2.53788549  0.24900125 -1.47544721 -1.2312115
   1.98112263  0.08341751]]
K
 [[-1.20165243e+00  6.92417656e-01 -5.66531887e-01  8.93579938e-01
   7.71733390e-01 -1.12475294e+00 -1.20605494e+00 -3.53952221e-01]
 [ 1.70565507e+00  1.32475072e+00 -1.16204025e-03 -2.04520927e-01
  -7.20643293e-02  2.80607587e-01  1.41351200e+00  8.31586155e-01]
 [-9.10301502e-01  6.72669461e-01 -1.09046468e+00  7.36874043e-01
   1.76078158e+00 -8.96642804e-01  2.70217759e-01 -4.76436881e-01]
 [ 5.51093949e-01 -2.49503709e-01 -1.55999448e+00 -2.30084179e-01
  -3.26128911e-01 -3.16256719e-01  1.68074362e+00 -1.17588938e+00]]
V
 [[-0.01790648 -0.64568823  1.03114358 -2.4772107  -1.0

In [10]:
np.matmul(q, k.T)


array([[-3.13433751,  0.55443147, -2.14408983,  1.52142095],
       [-0.51392765, -1.66402986, -2.8660383 , -5.69921933],
       [ 0.73519018,  1.87030002, -0.11482433,  0.05326988],
       [-1.1401601 ,  3.3258106 ,  1.46753517,  8.28756391]])

In [11]:
new = np.random.randn(10000000, d_v)
new.var()

0.9998819693276478

In [12]:
q.var(), k.var(), np.matmul(q, k.T).var()


(1.0528345952551859, 0.8863911357060673, 9.250403568268382)

In [13]:
scaled = np.matmul(q, k.T) / math.sqrt(d_k)
q.var(), k.var(), scaled.var()

(1.0528345952551859, 0.8863911357060673, 1.1563004460335478)

In [18]:
scaled

array([[-1.10815565,  0.19602113, -0.75805023,  0.53790353],
       [-0.18170086, -0.5883234 , -1.01329756, -2.01497832],
       [ 0.25992898,  0.66125092, -0.04059653,  0.01883375],
       [-0.40310747,  1.17585162,  0.51885204,  2.93009632]])

In [16]:

mask = np.tril(np.ones( (L, L) ))
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [17]:
scaled + mask

array([[-0.10815565,  0.19602113, -0.75805023,  0.53790353],
       [ 0.81829914,  0.4116766 , -1.01329756, -2.01497832],
       [ 1.25992898,  1.66125092,  0.95940347,  0.01883375],
       [ 0.59689253,  2.17585162,  1.51885204,  3.93009632]])

In [19]:

mask[mask == 0] = -np.infty
mask[mask == 1] = 0

In [20]:
scaled + mask

array([[-1.10815565,        -inf,        -inf,        -inf],
       [-0.18170086, -0.5883234 ,        -inf,        -inf],
       [ 0.25992898,  0.66125092, -0.04059653,        -inf],
       [-0.40310747,  1.17585162,  0.51885204,  2.93009632]])

In [25]:
def softmax(x):
  return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T
     

attention = softmax(scaled + mask)

In [26]:
attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.60027775, 0.39972225, 0.        , 0.        ],
       [0.30919288, 0.46187174, 0.22893538, 0.        ],
       [0.02747848, 0.13326803, 0.06908674, 0.77016675]])

In [27]:

new_v = np.matmul(attention, v)
new_v

array([[-0.01790648, -0.64568823,  1.03114358, -2.4772107 , -1.01925184,
        -0.11891081,  0.38383324, -0.08683348],
       [ 0.5624772 , -0.58881513,  1.15210345, -1.37220391, -0.37746556,
         0.32376545,  0.21010447,  0.09791406],
       [ 1.12468664, -0.17437473,  1.11395617, -0.73381591, -0.33206693,
         0.34656391,  0.2246673 , -0.21580364],
       [-0.94864034, -0.55495282,  0.06539455, -0.67342586,  0.96213442,
        -0.25787079, -1.54516812,  0.92828307]])

In [28]:
def softmax(x):
  return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T

def scaled_dot_product_attention(q, k, v, mask=None):
  d_k = q.shape[-1]
  scaled = np.matmul(q, k.T) / math.sqrt(d_k)
  if mask is not None:
    scaled = scaled + mask
  attention = softmax(scaled)
  out = np.matmul(attention, v)
  return out, attention
     

values, attention = scaled_dot_product_attention(q, k, v, mask=mask)
print("Q\n", q)
print("K\n", k)
print("V\n", v)
print("New V\n", values)
print("Attention\n", attention)

Q
 [[ 1.06265353 -1.04154709  0.31903798 -1.97891434  0.37866731 -0.49480558
   0.14147369 -0.38059492]
 [ 0.15804184 -0.53026353  0.368618   -1.09082603  0.56294831  1.03452456
  -2.03631891  1.41276778]
 [ 1.09129202  0.52080012 -0.24924261 -0.45375836 -0.18950295 -1.42962098
  -0.5359392   0.44610144]
 [ 0.4854946  -0.0640484  -2.53788549  0.24900125 -1.47544721 -1.2312115
   1.98112263  0.08341751]]
K
 [[-1.20165243e+00  6.92417656e-01 -5.66531887e-01  8.93579938e-01
   7.71733390e-01 -1.12475294e+00 -1.20605494e+00 -3.53952221e-01]
 [ 1.70565507e+00  1.32475072e+00 -1.16204025e-03 -2.04520927e-01
  -7.20643293e-02  2.80607587e-01  1.41351200e+00  8.31586155e-01]
 [-9.10301502e-01  6.72669461e-01 -1.09046468e+00  7.36874043e-01
   1.76078158e+00 -8.96642804e-01  2.70217759e-01 -4.76436881e-01]
 [ 5.51093949e-01 -2.49503709e-01 -1.55999448e+00 -2.30084179e-01
  -3.26128911e-01 -3.16256719e-01  1.68074362e+00 -1.17588938e+00]]
V
 [[-0.01790648 -0.64568823  1.03114358 -2.4772107  -1.0